# 【图像分类】使用飞桨全流程开发工具PaddleX实现垃圾分类任务

## 一、前言

* 随着人们保护环境的意识逐渐增强，垃圾分类成为每一个人的共识。但是繁杂的垃圾种类太难记了，于是自己想使用PP开发一个模型能替我完成垃圾分类的工具。但是AI Studio实现垃圾分类的项目比比皆是，于是我想到使用PaddleX来完成这个任务。
* 本项目粗暴地将垃圾按人们最熟悉的标准分为可回收垃圾、餐余垃圾、有害垃圾和其他垃圾四种，使用集飞桨核心框架、模型库、工具及组件等深度学习开发所需全部能力于一身的PaddleX实现垃圾的图像分类任务。
* 本项目的数据集和模型可以根据下面的提示操作换成大家自己的，然后快速实现自己图像分类项目的开发。

## 二、准备工作


In [1]:
# 下面的命令用于重新运行项目时删除前一次的残留文件
#!rm -rf output
#!rm -rf work/*
#!rm -rf inference_model

### 安装依赖

In [6]:
# 从百度镜像安装PaddleX
!pip install paddlex -i https://mirror.baidu.com/pypi/simple

### 关于数据

> 由于自己太懒了，所以本项目的数据集将数据集[垃圾分类图片](https://aistudio.baidu.com/aistudio/datasetdetail/35095)魔改成四类，生成四个文件夹放在数据集根目录下。

> 大家可以跟着下面的操作指引，换成自己的数据集以及对自己的数据集进行数据增强

In [2]:
# 解压数据集，放在work下使其可以永久保存（如果换数据集源记得更改下面的路径）
!unzip -oq /home/aistudio/data/data104285/Waste.zip -d work/

In [4]:
# 查看数据集文件结构
!tree work/Waste -L 1

work/Waste
├── FoodWaste
├── HarmfulWaste
├── OtherWaste
└── RecyclableWaste

4 directories, 0 files


In [7]:
# 使用PaddleX划分数据集
# API说明 https://paddlex.readthedocs.io/zh_CN/release-1.3/data/annotation/classification.html
# 下列参数dataset_dir后面可改成自己的数据集路径（绝对路径）
!paddlex --split_dataset --format ImageNet --dataset_dir work/Waste --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Dataset Split Done.
Train samples: 10084
Eval samples: 2879
Test samples: 1439
Split files saved in work/Waste


In [8]:
import paddlex as pdx
from paddlex.cls import transforms
# 数据增强：定义训练和验证时的transforms
# API说明 https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/cls_transforms.html
train_transforms = transforms.Compose([
    # 图像预处理代码：随机水平翻转、随机垂直翻转、随机旋转、标准化
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotate(),
    transforms.Normalize()
])

eval_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotate(),
    transforms.Normalize()
])

# 数据集读取
# API说明 https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/datasets.html
# path为数据集所在目录路径，可改成自己的数据集路径
path = 'work/Waste'
train_dataset = pdx.datasets.ImageNet(
                    data_dir= path,
                    file_list= path + '/train_list.txt',
                    label_list= path + '/labels.txt',
                    transforms=train_transforms)
eval_dataset = pdx.datasets.ImageNet(
                    data_dir= path,
                    file_list= path + '/val_list.txt',
                    label_list= path + '/labels.txt',
                    transforms=eval_transforms)

2021-08-15 15:02:59 [INFO]	Starting to read file list from dataset...
2021-08-15 15:03:00 [INFO]	10084 samples in file work/Waste/train_list.txt
2021-08-15 15:03:00 [INFO]	Starting to read file list from dataset...
2021-08-15 15:03:00 [INFO]	2879 samples in file work/Waste/val_list.txt


## 三、模型选择

这里选择移动端模型MobileNetV3作为开发模型

> MobileNetV3是在MobileNetV2的基础上提出的网络，其计算量小、参数少，相比其他轻量级网络，依然取得了较好的成绩；同时以ResNeXt101_32x16d_wsl为teacher模型，运用SSLD（简单的半监督标签知识蒸馏）方式蒸馏出MobileNetV3_large模型，作为预训练模型；相对比原有的MobileNetV3预训练模型，在参数量不变的情况下，MobileNetV3_ssld预训练模型在ImageNet数据集上的精度提升3%，有助于用户进一步提升在自定义数据集上模型训练的效果。

In [9]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### 模型训练
> 注：`model.train()`每次运行只能使用一次，需要多次使用时得重启环境；设置`train_batch_size`默认是32，可以根据模型大小设置，但是设太大容易爆内存

In [ ]:
# API说明 https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/models/classification.html
num_classes = len(train_dataset.labels)
# model可根据上面链接换成别的模型（注意修改模型参数和model.train的模型输出路径）
model = pdx.cls.MobileNetV3_large_ssld(num_classes=num_classes)
# model.train参数可参考上面的链接
model.train(num_epochs=1,
            train_dataset=train_dataset,
            train_batch_size=16,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[6, 8],
            save_interval_epochs=1,
            learning_rate=0.00625,
            save_dir='output/mobilenetv3_large_ssld',
            use_vdl=True)

## 四、效果展示



In [2]:
import paddlex as pdx
# 模型载入（记得根据模型修改路径）
model = pdx.load_model('output/mobilenetv3_large_ssld/best_model')
# 使用数据集文件夹下test.txt中的一张图片进行预测，打印预测结果（需要根据数据集修改路径）
image_name = 'work/Waste/RecyclableWaste/img_11129.jpg'
result = model.predict(image_name)
print("Predict Result:", result)

2021-08-15 14:03:25 [INFO]	Model[MobileNetV3_large_ssld] loaded.
Predict Result: [{'category_id': 3, 'category': 'RecyclableWaste', 'score': 0.9998091}]


## 五、模型导出

In [7]:
# 模型路径需要根据自己模型的输出路径进行更改
!paddlex --export_inference --model_dir=output/mobilenetv3_large_ssld/best_model --save_dir=inference_model

## 六、补充说明

* 此版本只是用PaddleX跑通了图像分类任务，关于模型优化和后续部署的部分有待更新。
* 本项目参考PaddleX官方文档和官方AI Studio上的项目。
* 关于使用PaddleX的感受，本人觉得这个工具很适合各位有创意的开发者进行快速开发，大家可以使用该工具完成自己的创意。
* 关于本项目使用PaddleX遇到的问题多为内存溢出，大家可以尝试重启环境释放内存以解决问题
* 大家如果运行此项目有什么问题欢迎评论交流=v=

## 个人简介

> 作者：刘廷楠


> 东北大学秦皇岛分校2019级车辆工程本科生


> 感兴趣方向：计算机视觉、深度学习


> 我在[AI Studio](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/537248)上获得黄金等级，点亮8个徽章，来互关呀~ 